In [1]:
from qoc.state_transfer import simulation
import numpy as np
%load_ext memory_profiler

In [ ]:
pre=2*np.pi
N_=40
#totol number of time step
mem_ma_Nfixed=[[],[]]
#first is average value,secind is standard deivation
h_dims=np.array([100,150,200,250,300,350,400])
for dim in h_dims:
    mem_temp = %memit -r1 -i0.0001 -o simulation(1, dim, 6, 6 * pre, 3 * pre, -0.225 * pre, 0.1 * pre, 10, 40,"AG")
    mem_temp=np.array(mem_temp.mem_usage)
    mem_ma_Nfixed[0].append(np.average(mem_temp))
    mem_ma_Nfixed[1].append(np.std(mem_temp))

        

In [ ]:
pre=2*np.pi
N_=40
#totol number of time step
mem_ma_Nfixed=[[],[]]
#first is average value,secind is standard deivation
h_dims=np.array([100,150,200,250,300,350,400])
for dim in h_dims:
    mem_temp = %memit -r1 -i0.0001 -o simulation(1, dim, 6, 6 * pre, 3 * pre, -0.225 * pre, 0.1 * pre, 10, 40,"AD")
    mem_temp=np.array(mem_temp.mem_usage)
    mem_ma_Nfixed[0].append(np.average(mem_temp))
    mem_ma_Nfixed[1].append(np.std(mem_temp))

        